In [1]:
import os
os.chdir( "../")

In [2]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus = 8


In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])

metric_to_optimize="MAP"

df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [5]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [7]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2136 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2128 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2180 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2196 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2202 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2173 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2187 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2162 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [8]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.Recommender_import_list import *
import optuna

In [9]:
# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

nmf_name = "NMF8-FOLD"
knn_name = "itemKNN8-FOLD"
slim_name = "SLIM_EN8-FOLD"
rp3beta_name = "RP3b8-FOLD"
p3alpha_name = "P3a8-FOLD"
hybrid_name = "HYBRID8-FOLD"
#nmf_study = optuna.load_study(study_name=nmf_name, storage=storage)
#knn_study = optuna.load_study(study_name=knn_name, storage=storage)
#slim_study = optuna.load_study(study_name=slim_name, storage=storage)
#rp3beta_study = optuna.load_study(study_name=rp3beta_name, storage=storage)
#p3alpha_study = optuna.load_study(study_name=p3alpha_name, storage=storage)
#hybrid_study = optuna.load_study(study_name=hybrid_name, storage=storage)

#studies = [nmf_study, knn_study, slim_study, rp3beta_study, p3alpha_study]
#studies = [knn_study, slim_study, rp3beta_study, p3alpha_study]
#recommenders_list = ["Hybrid", "itemKNN", "RP3beta", "P3alpha", "NMF", "SLIM_EN", "TopPop"]
    

In [10]:
metric_to_optimize = "MAP"

In [11]:
p3a_recomennders = []
rp3b_recomennders = []
slimen_recommenders = []
knn_recommenders = []
slim_recommenders = []

for i in range(len(seeds)):

    p3a_single = P3alphaRecommender(urm_train_validation[i])
    p3a_single.fit(**{'alpha':0.740243876372529,'topK':41})
    p3a_recomennders.append(p3a_single)

    rp3beta = RP3betaRecommender(urm_train_validation[i])
    rp3beta.fit(**{'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933, 'normalize_similarity': True})
    rp3b_recomennders.append(rp3beta)

    knn_single = ItemKNNCFRecommender(urm_train_validation[i])
    knn_single.fit(**{'topK': 19, 'shrink': 28, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.0, 'tversky_beta': 1.3672226785339947})
    knn_recommenders.append(knn_single)

    slim_single = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train_validation[i])
    slim_single.fit(workers=12,**{'alpha':0.0008361537677496276,'topK':3334, 'l1_ratio':0.09976758088492818})
    slim_recommenders.append(slim_single)

P3alphaRecommender: URM Detected 226 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 90 ( 0.4%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8054.34 column/sec. Elapsed time 2.76 sec
RP3betaRecommender: URM Detected 226 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 90 ( 0.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7548.28 column/sec. Elapsed time 2.94 sec
ItemKNNCFRecommender: URM Detected 226 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 90 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 10779.62 column/sec. Elapsed time 2.06 sec
SLIMElasticNetRecommender: URM Detected 226 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 90 ( 0.4%) items with no interactions.


100%|█████████▉| 22216/22222 [00:30<00:00, 735.32it/s] 


P3alphaRecommender: URM Detected 240 ( 1.9%) users with no interactions.
P3alphaRecommender: URM Detected 111 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8173.69 column/sec. Elapsed time 2.72 sec
RP3betaRecommender: URM Detected 240 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 111 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8338.06 column/sec. Elapsed time 2.67 sec
ItemKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 111 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12813.77 column/sec. Elapsed time 1.73 sec
SLIMElasticNetRecommender: URM Detected 240 ( 1.9%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 111 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:29<00:00, 748.89it/s] 


P3alphaRecommender: URM Detected 198 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 118 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8097.56 column/sec. Elapsed time 2.74 sec
RP3betaRecommender: URM Detected 198 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 118 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7572.39 column/sec. Elapsed time 2.93 sec
ItemKNNCFRecommender: URM Detected 198 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 118 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 11507.11 column/sec. Elapsed time 1.93 sec
SLIMElasticNetRecommender: URM Detected 198 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 118 ( 0.5%) items with no interactions.


...

100%|█████████▉| 22216/22222 [00:29<00:00, 741.70it/s] 


P3alphaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 115 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8166.26 column/sec. Elapsed time 2.72 sec
RP3betaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 115 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7662.98 column/sec. Elapsed time 2.90 sec
ItemKNNCFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 11215.90 column/sec. Elapsed time 1.98 sec
SLIMElasticNetRecommender: URM Detected 216 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 115 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:31<00:00, 706.13it/s] 


P3alphaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 108 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8018.20 column/sec. Elapsed time 2.77 sec
RP3betaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 108 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7534.51 column/sec. Elapsed time 2.95 sec
ItemKNNCFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 108 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 11183.55 column/sec. Elapsed time 1.99 sec
SLIMElasticNetRecommender: URM Detected 216 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 108 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:30<00:00, 740.19it/s] 


In [14]:
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender


hybrids = []

for i in range(len(seeds)):
    hybrid = GeneralizedLinearHybridRecommender(urm_train_validation[i], recommenders=[p3a_recomennders[i], rp3b_recomennders[i], knn_recommenders[i], slim_recommenders[i]])
    hybrids.append(hybrid)

ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 226 ( 1.8%) users with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 90 ( 0.4%) items with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 240 ( 1.9%) users with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 111 ( 0.5%) items with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 198 ( 1.6%) users with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 118 ( 0.5%) items with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 216 ( 1.7%) users with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 121 ( 0.5%) items with no interactions.
ImplicitALSP3alphaRP3betaItemKNNCFSLIMEla

In [15]:
hybrid_name = "HYBRID-NO_MF_ATALL-8-FOLD"
hybrid_study = optuna.create_study(study_name=hybrid_name, direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

def objective(trial):

    params = {
        #"w_ials": trial.suggest_float("w_ials", 0, 1),
        "w_p3a": trial.suggest_float("w_p3a", 0, 1),
        "w_rp3b": trial.suggest_float("w_rp3b", 0, 1),
        "w_knn": trial.suggest_float("w_knn", 0, 1),
        "w_slim": trial.suggest_float("w_slim", 0, 1),
    }

    alphas = [params["w_p3a"], params["w_rp3b"], params["w_knn"], params["w_slim"]]

    MAP = 0

    for i in range(len(seeds)):
        hybrids[i].fit(alphas=alphas)
        MAP += evaluator_test[i].evaluateRecommender(hybrids[i])[0][metric_to_optimize]

    average_score = MAP / len(seeds)
    return average_score

hybrid_study.enqueue_trial( {'w_p3a': 0.06535815757959068, 'w_rp3b': 0.9396285725235661, 'w_knn': 0.054706364180647754, 'w_slim': 0.9086719842062518})
hybrid_study.optimize(objective, n_trials=250)

[I 2024-01-09 14:40:46,261] A new study created in RDB with name: HYBRID-NONMF-8-FOLD


EvaluatorHoldout: Processed 10502 (100.0%) in 7.33 sec. Users per second: 1433
EvaluatorHoldout: Processed 10510 (100.0%) in 7.66 sec. Users per second: 1373
EvaluatorHoldout: Processed 10458 (100.0%) in 7.31 sec. Users per second: 1431
EvaluatorHoldout: Processed 10442 (100.0%) in 6.72 sec. Users per second: 1555
EvaluatorHoldout: Processed 10436 (100.0%) in 6.92 sec. Users per second: 1508
EvaluatorHoldout: Processed 10465 (100.0%) in 7.54 sec. Users per second: 1388
EvaluatorHoldout: Processed 10451 (100.0%) in 7.71 sec. Users per second: 1356
EvaluatorHoldout: Processed 10476 (100.0%) in 7.43 sec. Users per second: 1409


[I 2024-01-09 14:41:59,058] Trial 0 finished with value: 0.05181687986854874 and parameters: {'w_ials': 0.0, 'w_p3a': 0.06535815757959068, 'w_rp3b': 0.9396285725235661, 'w_knn': 0.054706364180647754, 'w_slim': 0.9086719842062518}. Best is trial 0 with value: 0.05181687986854874.


EvaluatorHoldout: Processed 10502 (100.0%) in 7.55 sec. Users per second: 1390
EvaluatorHoldout: Processed 10510 (100.0%) in 7.75 sec. Users per second: 1356
EvaluatorHoldout: Processed 10458 (100.0%) in 7.48 sec. Users per second: 1399
EvaluatorHoldout: Processed 10442 (100.0%) in 7.23 sec. Users per second: 1444
EvaluatorHoldout: Processed 10436 (100.0%) in 7.59 sec. Users per second: 1376
EvaluatorHoldout: Processed 10465 (100.0%) in 7.54 sec. Users per second: 1388
EvaluatorHoldout: Processed 10451 (100.0%) in 7.62 sec. Users per second: 1371
EvaluatorHoldout: Processed 10476 (100.0%) in 7.67 sec. Users per second: 1366


[I 2024-01-09 14:43:13,122] Trial 1 finished with value: 0.04853620331139224 and parameters: {'w_ials': 0.9583113241517032, 'w_p3a': 0.21820393840212726, 'w_rp3b': 0.47953420070653974, 'w_knn': 0.4712549049247361, 'w_slim': 0.03607280320442463}. Best is trial 0 with value: 0.05181687986854874.


EvaluatorHoldout: Processed 10502 (100.0%) in 7.81 sec. Users per second: 1345
EvaluatorHoldout: Processed 10510 (100.0%) in 7.64 sec. Users per second: 1376
EvaluatorHoldout: Processed 10458 (100.0%) in 7.31 sec. Users per second: 1432
EvaluatorHoldout: Processed 10442 (100.0%) in 7.02 sec. Users per second: 1487
EvaluatorHoldout: Processed 10436 (100.0%) in 7.24 sec. Users per second: 1441
EvaluatorHoldout: Processed 10465 (100.0%) in 7.58 sec. Users per second: 1380
EvaluatorHoldout: Processed 10451 (100.0%) in 7.62 sec. Users per second: 1371
EvaluatorHoldout: Processed 10476 (100.0%) in 7.47 sec. Users per second: 1402


[I 2024-01-09 14:44:26,334] Trial 2 finished with value: 0.0498238355127359 and parameters: {'w_ials': 0.023090788675259155, 'w_p3a': 0.7862469992384116, 'w_rp3b': 0.9156161019716993, 'w_knn': 0.8572293212080037, 'w_slim': 0.30831825372760646}. Best is trial 0 with value: 0.05181687986854874.


EvaluatorHoldout: Processed 10502 (100.0%) in 7.71 sec. Users per second: 1362
EvaluatorHoldout: Processed 10510 (100.0%) in 7.55 sec. Users per second: 1392
EvaluatorHoldout: Processed 10458 (100.0%) in 7.28 sec. Users per second: 1437
EvaluatorHoldout: Processed 10442 (100.0%) in 7.18 sec. Users per second: 1454
EvaluatorHoldout: Processed 10436 (100.0%) in 7.67 sec. Users per second: 1360
EvaluatorHoldout: Processed 10465 (100.0%) in 7.89 sec. Users per second: 1326
EvaluatorHoldout: Processed 10451 (100.0%) in 7.74 sec. Users per second: 1351
EvaluatorHoldout: Processed 10476 (100.0%) in 7.93 sec. Users per second: 1321


[I 2024-01-09 14:45:40,888] Trial 3 finished with value: 0.048364570782358375 and parameters: {'w_ials': 0.9343434977770495, 'w_p3a': 0.4196527124836651, 'w_rp3b': 0.6375105498784669, 'w_knn': 0.25603848693429854, 'w_slim': 0.0697015744675884}. Best is trial 0 with value: 0.05181687986854874.


EvaluatorHoldout: Processed 10502 (100.0%) in 7.74 sec. Users per second: 1357
EvaluatorHoldout: Processed 10510 (100.0%) in 7.67 sec. Users per second: 1370
EvaluatorHoldout: Processed 10458 (100.0%) in 7.28 sec. Users per second: 1436
EvaluatorHoldout: Processed 10442 (100.0%) in 7.25 sec. Users per second: 1440
EvaluatorHoldout: Processed 10436 (100.0%) in 7.72 sec. Users per second: 1351
EvaluatorHoldout: Processed 10465 (100.0%) in 7.85 sec. Users per second: 1333
